In [1]:
import numpy as np

In [2]:
def encode(labels):
    return (labels -100)/300
def decode(enc_labels):
    return (labels*300)+100

In [3]:
number_to_train = 8000
number_to_validate = 1000
number_to_test = 1000

In [4]:
#features_binned_orig = np.loadtxt("features_binned.csv")
#results_orig = np.loadtxt("results.csv")
#features_binned_orig = np.loadtxt("features_binned.csv")
#results_orig = np.loadtxt("results.csv")

features_binned_orig = np.load("features_binned.npy")
results_orig = np.load("results.npy")

In [5]:
#features_binned = features_binned_orig.reshape([202,50,50,100,1])
features_binned = features_binned_orig.reshape([10000,15,15,25,1])
#features_binned = np.rollaxis(features_binned,0,5)
results_enc = encode(results_orig) 

print "features_binned.shape : ", features_binned.shape
print "results.shape : ", results_enc.shape

features_binned.shape :  (10000, 15, 15, 25, 1)
results.shape :  (10000,)


In [8]:
features_binned_orig.shape
#features_binned[:number_to_train,:,:,:,:].shape


(10000, 15, 15, 25)

In [6]:
from keras.layers import Input, Dense, Dropout, Flatten 
from keras.layers import Conv3D, MaxPooling3D
from keras.models import Model

# This returns a tensor
#inputs = Input(shape=(50,50,100,))
inputs = Input(shape=(50,50,100,1,)) 
inputs = Input(shape=(15,15,25,1,)) 
#labels shape (,1)



x = Conv3D(8, (3, 3, 3), activation='relu')(inputs)
#x = MaxPooling3D(pool_size=(3,3,5))(x)
x = Conv3D(8, (3, 3, 5), activation='relu')(x)
x = Flatten()(x)




# a layer instance is callable on a tensor, and returns a tensor
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='tanh')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='mse',   #mean squared error
              metrics=[])
model.fit(features_binned[:number_to_train,:,:,:,:], results_enc[:number_to_train], epochs=10, batch_size=25)  # starts training

Using Theano backend.


Epoch 1/10
8000/8000 [==============================] - 2798s - loss: 0.0251  
Epoch 2/10
8000/8000 [==============================] - 2525s - loss: 0.0048  
Epoch 3/10
8000/8000 [==============================] - 2238s - loss: 0.0027  
Epoch 4/10
8000/8000 [==============================] - 41887s - loss: 0.0019   
Epoch 5/10
8000/8000 [==============================] - 2071s - loss: 0.0014  
Epoch 6/10
8000/8000 [==============================] - 2261s - loss: 0.0012  
Epoch 7/10
8000/8000 [==============================] - 2178s - loss: 9.5960e-04  
Epoch 8/10
8000/8000 [==============================] - 2088s - loss: 8.9304e-04  
Epoch 9/10
8000/8000 [==============================] - 2012s - loss: 7.6597e-04  
Epoch 10/10
8000/8000 [==============================] - 2038s - loss: 6.7409e-04  


In [ ]:
score = model.evaluate(features_binned[number_to_train:number_to_train+number_to_test,:,:,:], results_enc[number_to_train:number_to_train+number_to_test], batch_size=25)
print score
# past 0.0154 for 8 convolutional filters.

2000/2000 [==============================] - 165s   
0.0154846393591


In [14]:
predicted = model.predict(features_binned[number_to_train:number_to_train+number_to_test,:])

In [15]:
print predicted, results_enc[number_to_train:number_to_train+number_to_test]

[[ 0.12312661]
 [-0.0946205 ]
 [-0.0349816 ]
 ..., 
 [-0.0349816 ]
 [-0.03366851]
 [-0.18556595]] [ 0.00427553 -0.13653037 -0.08732112 ...,  0.02201658 -0.0313603
 -0.11749169]


In [16]:
b = np.c_[results_enc[number_to_train:number_to_train+number_to_test], predicted]
for j in b:
    print j


[ 0.00427553  0.12312661]
[-0.13653037 -0.0946205 ]
[-0.08732112 -0.0349816 ]
[ 0.03918332  0.01507268]
[ 0.04841062  0.00892746]
[ 0.02084904 -0.0349816 ]
[-0.00548944 -0.01152889]
[ 0.01484788  0.07738288]
[ 0.05970674 -0.0349816 ]
[-0.20724522 -0.14055185]
[ 0.05440949  0.02182931]
[-0.18050186 -0.18266457]
[-0.28414229 -0.0349816 ]
[ 0.0411041 -0.0349816]
[ 0.12731889  0.0539727 ]
[-0.04722017 -0.0349816 ]
[ 0.1591474   0.09915946]
[ 0.09261331  0.0954761 ]
[ 0.35313614 -0.0349816 ]
[ 0.25571762 -0.0349816 ]
[-0.05049794 -0.0349816 ]
[ 0.07460424  0.0768964 ]
[-0.0081176  -0.05046279]
[ 0.21791206  0.08632564]
[-0.10945967 -0.04613041]
[ 0.02617983 -0.05564908]
[-0.2047828  -0.28687507]
[ 0.11235085  0.0065296 ]
[-0.02329555 -0.22408627]
[ 0.23847676  0.20524134]
[ 0.1159372 -0.0349816]
[-0.11788933 -0.27531248]
[ 0.35879777  0.2379452 ]
[-0.32240942 -0.0349816 ]
[ 0.08524777 -0.0349816 ]
[ 0.05413497  0.00590697]
[ 0.05485958 -0.0349816 ]
[-0.06291421 -0.0349816 ]
[ 0.03499419 -0.

[-0.13328314 -0.0349816 ]
[-0.00559432 -0.0349816 ]
[ 0.03944128  0.12133817]
[-0.05874414 -0.06789706]
[-0.42234143 -0.42731321]
[ 0.07544279  0.05372384]
[-0.08854174 -0.18710279]
[ 0.12094476 -0.0349816 ]
[-0.29784348 -0.33399528]
[-0.05579773 -0.0349816 ]
[ 0.14266082  0.14566018]
[-0.18976103 -0.0349816 ]
[-0.0381599  -0.07847354]
[-0.32661281 -0.29991493]
[ 0.02294902 -0.06245626]
[-0.1135157 -0.0349816]
[ 0.13960687  0.1062549 ]
[-0.05156315  0.0034192 ]
[-0.00328819 -0.10521872]
[ 0.0273365  -0.07858162]
[-0.03424663 -0.11401066]
[ 0.15568615  0.06561178]
[ 0.02864819  0.03180307]
[ 0.12045514  0.11472678]
[ 0.4642336   0.25880548]
[-0.26246805 -0.0349816 ]
[-0.10661359 -0.08904889]
[ 0.05426653  0.08156885]
[-0.00074037 -0.0698131 ]
[-0.14975806 -0.0349816 ]
[-0.04833458 -0.0349816 ]
[-0.29279226 -0.0349816 ]
[-0.40858708 -0.45027214]
[ 0.03492336  0.05062504]
[ 0.31523985 -0.0349816 ]
[ 0.44830079 -0.0349816 ]
[-0.09344477 -0.20987388]
[ 0.22183599  0.24687846]
[ 0.10911951  

[ 0.37377549  0.3496007 ]
[-0.23998848 -0.0349816 ]
[ 0.16454945  0.0622457 ]
[ 0.38724193  0.31195569]
[ 0.08439206  0.14112046]
[ 0.05562778 -0.0349816 ]
[ 0.2189604 -0.0349816]
[ 0.24897581 -0.0349816 ]
[ 0.28025451  0.19923142]
[ 0.38350969  0.40291578]
[-0.01568951  0.03578864]
[-0.05254068 -0.08575614]
[-0.32072887 -0.52940941]
[-0.25490146 -0.18415025]
[ 0.07891807  0.04425345]
[ 0.33950382  0.26619208]
[-0.02167501 -0.0349816 ]
[-0.21580088 -0.07974498]
[-0.06552756 -0.02630609]
[-0.19703665 -0.0349816 ]
[-0.03769611 -0.0349816 ]
[ 0.19010753  0.30048448]
[-0.29169684 -0.34959149]
[-0.18749075 -0.18741688]
[ 0.11190234  0.09713918]
[-0.45357502 -0.59432793]
[-0.4810641 -0.0349816]
[ 0.01042105 -0.0349816 ]
[-0.06046551 -0.02170015]
[ 0.20892122  0.18936825]
[-0.05067527 -0.11548275]
[-0.03393869 -0.0349816 ]
[ 0.11538059 -0.0349816 ]
[-0.03012379 -0.0349816 ]
[ 0.00744647  0.0136895 ]
[ 0.04991822 -0.08840434]
[-0.05321474 -0.03561222]
[ 0.17260151 -0.0349816 ]
[ 0.08576725 -0.